## Instalar e importar módulos

In [1]:
#Beautiful Soup de bs4 y requests sirven para WebScraping (ya instalados con Anaconda)
#Selenium sirve para automatizar el navegador(desde Anaconda Prompt ejecutar: "pip install selenium")
#PyAutoGUI sirve para automatizar movimientos de mouse, clicks, teclado ("pip install pyautogui")
#pyperclip para trabajar con portapapeles ("pip install pyperclip")
#pandas para armar el dataframe, time para hacer pausas entre comandos

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import pyautogui as pg
import pyperclip as pc
import time

In [3]:
#Parsers de Scraping: lxml y html5lib (ya instalados con Anaconda)
#Webdriver de Chrome: Ir a "https://sites.google.com/a/chromium.org/chromedriver/downloads" para descargar un WebDriver
#para usarlo con Selenium  
#Revisar que sea la misma versión, el mío es ChromeDriver 83.0.4103.39 para Windows, para revisar la versión correcta
#ir a las opciones a la esquina superior derecha de Chrome > Help > About Google Chrome

## Entrar a PowerCampus > Clases Virtuales con Selenium

In [3]:
browser = webdriver.Chrome("C:\Program Files (x86)\chromedriver83.exe") #Acá puse el driver
browser.get("https://autoservicio2.up.edu.pe/ss/Home.aspx")
barra_user = browser.find_element_by_id("ctl00_ucUserLogin_lvLoginUser_ucLoginUser_lcLoginUser_UserName")
barra_password = browser.find_element_by_id("ctl00_ucUserLogin_lvLoginUser_ucLoginUser_lcLoginUser_Password")
barra_user.send_keys("*") #Acá poner su usuario UP
barra_password.send_keys("*") #Acá la contraseña
barra_password.send_keys(Keys.ENTER)
boton_clases = browser.find_element_by_link_text("Clases")
boton_clases.send_keys(Keys.ENTER)
boton_clasesvirtuales = browser.find_element_by_link_text("Clases Virtuales")
boton_clasesvirtuales.send_keys(Keys.ENTER)
pagina_base = browser.current_url

## Navegar cursos y obtener links con Beautiful Soup o requests

### Videos de un curso

In [5]:
boton_desplegable = browser.find_element_by_id("select2-searchCursos-container")
boton_desplegable.click()
busqueda = browser.find_element_by_class_name("select2-search__field")
busqueda_desplegable = browser.find_element_by_xpath('//*[@id="ctl00_Html1"]/body/span/span/span[1]/input')
busqueda_desplegable.send_keys("macroe") #Nombre del curso
busqueda_desplegable.send_keys(Keys.ENTER)

In [7]:
#Scraping
time.sleep(1.8)
pg.hotkey('ctrl','shift','i')
time.sleep(1.5)
for i in range(0,2):
    pg.press('up')
pg.hotkey('shift','f10')
time.sleep(1)
for i in range(0,5):
    pg.press('down')
pg.press('right')
for i in range(0,2):
    pg.press('down')
pg.press('enter')
html = pc.paste()
soup = BeautifulSoup(html, "lxml") #soup es el objeto, lmxl es el parser
#Ahora lo guardo en un archivo 'html.html'
with open('html.html', "wb") as file:
    file.write(soup.prettify("utf-8"))

## Obtener nombres de cursos

In [8]:
cursoslista = soup.find('select', {"id":"searchCursos"})
cursostexto = cursoslista.contents
cursos=[] #Acá están los nombres de todos los cursos UP
for i in range(1,len(cursostexto)):
    cursos.append(cursostexto[i].text)

## Obtener cabeceras y datos

### Cabeceras

In [11]:
tabla = soup.find('table', {"class":"table","id":"tablaResultado"})
cabeceraslista = tabla.thead.tr
cabecerastexto = cabeceraslista.contents
cabeceras=[]
for i in range(1,len(cabecerastexto)-1,2):
    cabeceras.append(cabecerastexto[i].text)
cabeceras.append('Link2')

### Datos

In [12]:
datos = tabla.tbody
claseslista = datos.contents
curso=[]
seccion=[]
fecha=[]
nombreseccion=[]
duracion=[]
link=[] #Acá estarán los urls ca.bbcollab
link2=[] #Acá estarán los urls cloudfront
# claseslista[#].find_all('td') ubica clase número #
# El .text obtiene el contenido de la etiqueta; el a.get('href') obtiene el url del link

for i in range(0,len(claseslista)):
    curso.append((claseslista[i].find_all('td')[0]).text)
    seccion.append((claseslista[i].find_all('td')[1]).text)
    fecha.append((claseslista[i].find_all('td')[2]).text)
    nombreseccion.append((claseslista[i].find_all('td')[3]).text)
    duracion.append((claseslista[i].find_all('td')[4]).text)
    link.append((claseslista[i].find_all('td')[5]).a.get('href'))

#### Obtener los link2

In [42]:
#Esta subrutina es un while que espera hasta que aparezca una imagen en pantalla y mueve el cursor hacia ella
def esperaryencontrar(imagen):
    r=None
    while r is None:
        time.sleep(0.1)
        r=pg.locateCenterOnScreen(imagen)
        if r==None:
            print("Todavía no se encuentra su imagen %s" %imagen)
        else:
            pg.moveTo(r)

In [43]:
#Esta subrutina es un while que espera hasta que aparezca una imagen en pantalla y hace click en ella
def esperaryclickear(imagen):
    r2=None
    while r2 is None:
        time.sleep(0.1)
        r2=pg.locateCenterOnScreen(imagen)
        if r2==None:
            print("Todavía no se encuentra su imagen %s" %imagen)
        else:
            pg.click(r2)

In [ ]:
#Esta subrutina es un while que espera hasta que aparezca una imagen en pantalla y hace click derecho en ella
def esperaryclickearderecha(imagen):  
    r3=None
    while r3 is None:
        time.sleep(0.1)
        r3=pg.locateCenterOnScreen(imagen)
        if r3==None:
            print("Todavía no se encuentra su imagen %s" %imagen)
        else:
            pg.click(r3,button='right')

In [ ]:
browser = webdriver.Chrome("C:\Program Files (x86)\chromedriver83.exe")
for i in range(0,len(link)):
    browser.get(link[i])
    if i==0:
        pg.hotkey('ctrl','shift','i')
        pg.hotkey('winleft','up')
        esperaryclickear('Network.png')
        time.sleep(1)
        for i in range(2):
            pg.press('tab')
        for i in range(5):
            pg.press('right')
        pg.press('enter')
        for i in range(5):
            time.sleep(0.1)
            pg.press('tab')
        time.sleep(5)
    else:
        time.sleep(8)
    pg.press('down')
    time.sleep(1)
    pg.hotkey('shift','f10')
    time.sleep(1)
    pg.press('down')
    time.sleep(0.5)
    pg.press('enter')
    browser.switch_to.window(browser.window_handles[1])
    link2.append(browser.current_url)
    pg.hotkey('ctrl','w')
    browser.switch_to.window(browser.window_handles[0])

### Original idea de Scraping

In [ ]:
# #Scraping
# pedido = requests.get(pagina_curso) #pedido GET request
# pedido
# html = pedido.content #su contenido
# soup = BeautifulSoup(html, "lxml") #soup es el objeto, lmxl es el parser
# #Ahora lo guardo en un archivo 'pedido.html'
# with open('pedido.html', "wb") as file:
#     file.write(soup.prettify("utf-8"))
# #Este pedido cambia con el Javascript así que no sirve, por eso lo sacaré con PyAutoGUI

###  Original forma de obtener los link

In [342]:
# enlaces = browser.find_elements_by_xpath('.//a[text()="ingresar"]') #Este último es ubicar los enlaces por su XPATH
# urls = [] #Acá están los urls
# for n in enlaces:
#     urls.append(n.get_attribute('href'))

In [379]:
# browser = webdriver.Chrome("C:\Program Files (x86)\chromedriver83.exe")
# browser.get(link[1])
# pg.hotkey('ctrl','shift','i')
# time.sleep(2.5)
# for i in range(5):
#     pg.hotkey('shift','tab') #5 shift tab
# for i in range(3):
#     pg.press('right') #2 derecha
# # for i in range(10):
# #     pg.hotkey('shift','tab') #10 shift tab
# # pg.press('right') #1 derecha
# for i in range(22):
#     pg.press('tab') #20 derecha
# time.sleep(8) #wait?
# pg.press('up') #1 arriba
# pg.hotkey('shift','f10') #1 shift f10
# time.sleep(1)
# for i in range(10):
#     pg.press('up') #1 abajo
# pg.press('enter') #enter

### Prueba de obtener los link2 mediante su html

In [40]:
#Este código al parecer no funcionaba porque se tardaba en descargar el javscript de 'media' en 'network', por tanto  el souptemp.find('video') devolvía None en vez del url
# browser = webdriver.Chrome("C:\Program Files (x86)\chromedriver83.exe")
# link2=[]
# # for i in range(0,4):
# time.sleep(3)
# browser.get(link[6])
# # if i==0:
# pg.hotkey('ctrl','shift','i')
# time.sleep(2.5)
# for i in range(2):
#     pg.press('up')
# pg.hotkey('shift','f10')
# time.sleep(2.5)
# for i in range(5):
#     pg.press('down')
# pg.press('right')
# for i in range(2):
#     pg.press('down')
# pg.press('enter')
# htmltemp = pc.paste()
# souptemp = BeautifulSoup(htmltemp, "lxml")
# with open('temp.html', "wb") as file:
#     file.write(souptemp.prettify("utf-8"))
# tagurl = souptemp.find('video',{'class':'vjs-tech','style':'height: 100%; width: 100%;'})
# tagur1

# {'id':'playback-video-playback-video_html5_api'}
# link2.append(tagurl.get('src'))
# link2.append(tagurl.get('src'))
# link2
# souptemp.find('video',{"id":"playback-video-playback-video_html5_api"})

### Original forma de obtener los link2

In [41]:
#Este código sí funcionaba pero era inconsistente probablemente porque no reconocía bien la imagen 'Media.png'
# time.sleep(1.5)
# for i in range(0,len(link)):
#     browser.get(link[i])
#     if i == 0:
#         pg.hotkey('ctrl','shift','i') #Abrir las herramientas del desarrollador
#         pg.hotkey('winleft','up') #Maximizar la ventana
#         time.sleep(1.5)
#         pg.click(pg.locateCenterOnScreen('Network.png'))
#     time.sleep(10)
#     pg.click(pg.locateCenterOnScreen('Media.png'),button='right')
#     pg.click(pg.locateCenterOnScreen('Open.png'))
#     browser.switch_to.window(browser.window_handles[1])
#     link2.append(browser.current_url)
#     pg.hotkey('ctrl','w')
#     browser.switch_to.window(browser.window_handles[0])
# link2
# len(link2)

## Marco de datos con pandas y exportar como csv

In [39]:
datostot=[curso,seccion,fecha,nombreseccion,duracion,link,link2]
marco1 = pd.DataFrame()
for i in range(0,len(cabeceras)):
    marco1[cabeceras[i]]=datostot[i]
marco1

,Curso,Sección,Fecha,Nombre de la sesión,Duración,Link,Link2
0,Macroeconomía Financiera,A,2020-03-30,Sesión 1 - Introducción,1:49:44,https://ca.bbcollab.com/recording/db5cd8d1e0ba...,https://dfz4hf1hz1uri.cloudfront.net/content/6...
1,Macroeconomía Financiera,A,2020-04-02,Sesión 2 - Indicadores Macroeconómicos,1:44:43,https://ca.bbcollab.com/recording/84aa601400fc...,https://dfz4hf1hz1uri.cloudfront.net/content/7...
2,Macroeconomía Financiera,A,2020-04-04,Sesión 3 - Aplicaciones,1:48:12,https://ca.bbcollab.com/recording/b5fc4162436a...,https://dfz4hf1hz1uri.cloudfront.net/content/1...
3,Macroeconomía Financiera,A,2020-04-06,Sesión 4 - Macroeconomía Financiera,1:43:06,https://ca.bbcollab.com/recording/743a6402c461...,https://dfz4hf1hz1uri.cloudfront.net/content/3...
4,Macroeconomía Financiera,A,2020-04-09,Sesión 5 - Macroeconomía Financiera,2:00:12,https://ca.bbcollab.com/recording/643c9168fd35...,https://dfz4hf1hz1uri.cloudfront.net/content/6...
5,Macroeconomía Financiera,A,2020-04-11,Sesión 6 - Macroeconomía Financiera,1:53:46,https://ca.bbcollab.com/recording/0c626ce722b1...,https://dfz4hf1hz1uri.cloudfront.net/content/1...
6,Macroeconomía Financiera,A,2020-04-13,Sesión 7 - Macroeconomía Financiera,1:48:23,https://ca.bbcollab.com/recording/96240ea40664...,https://dfz4hf1hz1uri.cloudfront.net/content/2...
7,Macroeconomía Financiera,A,2020-04-16,Sesión 8 - Crecimiento Económico,0:36:38,https://ca.bbcollab.com/recording/1702d200b6ca...,https://dfz4hf1hz1uri.cloudfront.net/content/c...
8,Macroeconomía Financiera,A,2020-04-16,Sesión 8 - Crecimiento Económico,0:20:03,https://ca.bbcollab.com/recording/4b75b23049ab...,https://dfz4hf1hz1uri.cloudfront.net/content/5...
9,Macroeconomía Financiera,A,2020-04-18,Sesión 9 - PC1,0:03:05,https://ca.bbcollab.com/recording/1ef56a55a4db...,https://dfz4hf1hz1uri.cloudfront.net/content/b...


In [ ]:
marco1.to_csv("marco1.csv", index=True, header=True)